In [123]:

url = ""
with open("../app.env") as file:
    db_source = file.read()
    url = db_source.split("DB_SOURCE=\"")[1][:-1]

In [124]:
from pymongo import MongoClient

import time
client = MongoClient(url)
db = client.wnba

In [125]:
from nba_api.stats.endpoints import teaminfocommon
from nba_api.stats.endpoints import leaguestandingsv3

teamLookup = {}
xxx = leaguestandingsv3.LeagueStandingsV3(league_id=10, season="2022").get_normalized_dict()
for team in xxx["Standings"]:
    teaminfo = teaminfocommon.TeamInfoCommon(team["TeamID"], league_id=10).get_normalized_dict()["TeamInfoCommon"][0]
    team_data = {
        "teamID": teaminfo['TEAM_ID'],
        'name': teaminfo['TEAM_NAME'],
        'abbreviation': teaminfo['TEAM_ABBREVIATION'],
        "city": teaminfo["TEAM_CITY"],
    }
    teamLookup[teaminfo["TEAM_ABBREVIATION"]] = teaminfo["TEAM_ID"]
    # if team["abbreviation"] == "OKC":
    #     team_data["alternate_abbreviations"] = ["SEA"]
    # if team["abbreviation"] == "NOP":
    #     team_data["alternate_abbreviations"] = ["NOK", "NOH"]
    # if team["abbreviation"] == "BKN":
    #     team_data["alternate_abbreviations"] = ["NJN"]
    db.teams.update_many({"teamID": team_data['teamID']}, {"$set": team_data}, True)


In [126]:
def lookupTeam(abbreviation):
    return teamLookup[abbreviation]

In [127]:
from datetime import datetime
lastModifiedDate = datetime.now()
print(lastModifiedDate)

2022-06-28 09:39:56.159222


In [128]:
correctNames= {
    "Asia (AD) Durr": {"fn":"Asia", "ln":"Durr"},
    "Azurá Stevens": {"fn":"Azura", "ln":"Stevens"},
}

    

In [129]:
import requests
from nba_api.stats.endpoints import commonplayerinfo
from nba_api.stats.endpoints import playercareerstats
from nba_api.stats.library.parameters import PerModeSimple

In [130]:

# # add players to db
# #TODO: query all players from db at once instead of one at a time
# get_season="2022"x
# players_url="https://data.wnba.com/data/5s/v2015/json/mobile_teams/wnba/"+get_season+"/players/10_player_info.json"
# all_players = requests.get(players_url).json()["pls"]["pl"]
# attempt = 0
# sleeptime=5
# total_players = len(all_players)
# for attempt in range(5):
#     print("attempt: " + str(attempt))
#     count = 0
#     try:
#         for count in range(total_players):
#             player = all_players[count]
#             fn = player["fn"]
#             ln = player["ln"]
#             name = player["fn"] + " " + player["ln"]
#             if name in correctNames:
#                 save_name = correctNames[name]
#                 name = save_name["fn"] + " " + save_name["ln"]
#                 fn = save_name["fn"]
#                 ln = save_name["ln"]
#             # else:
#             #     continue
            
#             player_data = {
#                 "playerID":player["pid"],
#                 "name":name,
#                 "first_name":fn,
#                 "last_name":ln,
#                 "teamABR": player["ta"],
#                 "height":player["ht"],
#                 "weight":player["wt"],
#                 "position":player["pos"],
#                 "lastModifiedDate":lastModifiedDate,
#                 "seasons":[],
#             }
#             player_career = playercareerstats.PlayerCareerStats(player["pid"], per_mode36=PerModeSimple.per_game, league_id_nullable=10)
#             regular_seasons = player_career.get_normalized_dict()["SeasonTotalsRegularSeason"]
#             time.sleep(sleeptime)
#             for season in regular_seasons:
#                 player_data["seasons"].append(season["SEASON_ID"])
#                 if season["SEASON_ID"] == "2022-23":
#                     player_data["teamABR"] = season["TEAM_ABBREVIATION"]
#             # if player_data["weight"] == "" or player_data["height"] == "" or player_data["position"] == "":
#             #     info = commonplayerinfo.CommonPlayerInfo(player["pid"], league_id_nullable=10).get_normalized_dict()["CommonPlayerInfo"][0]
#             #     player_data["position"]=info["POSITION"]
#             #     player_data["height"]=info["HEIGHT"]
#             #     player_data["weight"]=int(info["WEIGHT"])
#             db.players.update_one({"playerID": player_data["playerID"]}, {"$set":player_data}, True)
#             count+=1
#             print(count, player_data["first_name"], player_data["last_name"])
#         break
#     except Exception as e:
#         print("failed on attempt: " + str(attempt))
#         print(e)
#         print(player_data)
#         time.sleep(sleeptime*10)
#     attempt+=1
        
    

In [131]:
def game_exists(games, game_id):
    for game in games:
        if game["gameID"] == game_id:
            if "lastModifiedDate" in game.keys():
                # last modified after quality assurance date
                if game["lastModifiedDate"] > datetime.strptime("2022-04-20T11:50:00", "%Y-%m-%dT%H:%M:%S"):
                    return True
    return False

In [132]:
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.endpoints import boxscoretraditionalv2
from nba_api.stats.endpoints import boxscoresummaryv2
from nba_api.stats.endpoints import boxscoreadvancedv2
from nba_api.stats.endpoints import boxscoremiscv2

sleeptime = .6

def get_game_data(g, count, playoffs):
    try:
        time.sleep(sleeptime)
        boxscorefinder = boxscoretraditionalv2.BoxScoreTraditionalV2(g["GAME_ID"]).get_normalized_dict()
        time.sleep(sleeptime)
        summary = boxscoresummaryv2.BoxScoreSummaryV2(g["GAME_ID"]).get_normalized_dict()["GameSummary"][0]
        time.sleep(sleeptime)
        advanced = boxscoreadvancedv2.BoxScoreAdvancedV2(g["GAME_ID"]).get_normalized_dict()
        time.sleep(sleeptime)
        misc = boxscoremiscv2.BoxScoreMiscV2(g["GAME_ID"]).get_normalized_dict()
        season = summary["SEASON"] + "-"+ str(int(summary["SEASON"][2:])+1)
        # print("got stats")
        #team stats
        # print(count, " ", "VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV")
        for i in range(2):
            game = boxscorefinder["TeamStats"][i]
            if i == 1:
                opponentgame = boxscorefinder["TeamStats"][0]
            else:
                opponentgame = boxscorefinder["TeamStats"][1]
            #if the team's game stats haven't been added already
            if game["PTS"] != None:
                # print("got team stats")
                home_or_away = ""
                if game["TEAM_ID"] == summary["HOME_TEAM_ID"]:
                    home_or_away = "home"
                elif game["TEAM_ID"] == summary["VISITOR_TEAM_ID"]:
                    home_or_away = "away"
                else:
                    print("Error: team id not found")
                    exit(0)
                opponent = opponentgame["TEAM_ID"]
                win_or_loss = "loss"
                if game["PTS"] > opponentgame["PTS"]:
                    win_or_loss = "win"
                team_game_data = {
                    "date": summary["GAME_DATE_EST"],
                    "margin": game["PTS"] - opponentgame["PTS"],
                    "points": game["PTS"],
                    "assists": game["AST"],
                    "rebounds": game["REB"],
                    "offensive_rebounds": game["OREB"],
                    "defensive_rebounds": game["DREB"],
                    "turnovers": game["TO"],
                    "steals": game["STL"],
                    "blocks": game["BLK"],
                    "three_pointers_attempted": game["FG3A"],
                    "three_pointers_made": game["FG3M"],
                    "defensive_rating": advanced["TeamStats"][i]["DEF_RATING"],
                    "defensive_rebound_percentage": advanced["TeamStats"][i]["DREB_PCT"],
                    "field_goal_percentage": game["FG_PCT"],
                    "field_goals_attempted": game["FGA"],
                    "field_goals_made": game["FGM"],
                    "gameID":game["GAME_ID"],
                    "home_or_away":home_or_away,
                    "offfensive_rebound_percentage": advanced["TeamStats"][i]["OREB_PCT"],
                    "opponent":opponent,
                    "opponent_assists": opponentgame["AST"],
                    "opponent_effective_field_goal_percentage": opponentgame["FG_PCT"],
                    "opponent_field_goals_attempted": opponentgame["FGA"],
                    "opponent_free_throws_attempted": opponentgame["FTA"],
                    "opponent_points": opponentgame["PTS"],
                    "opponent_rebounds": opponentgame["REB"],
                    "opponent_three_pointers_attempted": opponentgame["FG3A"],
                    "opponent_three_pointers_made": opponentgame["FG3M"],
                    "personal_fouls": misc["sqlTeamsMisc"][i]["PF"],
                    "personal_fouls_drawn": misc["sqlTeamsMisc"][i]["PFD"],
                    "plus_minus_per_hundred": ((int(game["PTS"])-int(opponentgame["PTS"]))/float(advanced["TeamStats"][i]["POSS"]))*100,
                    "possessions": advanced["TeamStats"][i]["POSS"],
                    "season": season,
                    "teamID": game["TEAM_ID"],
                    "win_or_loss":win_or_loss,
                    "playoffs": playoffs,
                    "lastModifiedDate":lastModifiedDate
                }
                # print("inserting team stats")
                db.teamgames.replace_one({"teamID": team_game_data["teamID"], "gameID":team_game_data["gameID"]}, team_game_data, True)
                print("\r ",count, game["TEAM_NAME"], " vs ", opponentgame["TEAM_NAME"], team_game_data["season"], team_game_data["gameID"], end="")
        #players stats
        for game in boxscorefinder["PlayerStats"]:
            # print("got player stats")
            #if the player played in the game, and the player exists with the provided season (averaged > 10 min in the season), and this player's game stats haven't been added already
            if game["MIN"] != None and db.players.count_documents({"playerID": game["PLAYER_ID"], "seasons":season}) > 0:
                # print("adding player stats")
                team1 = lookupTeam(g["MATCHUP"].split(" ")[0])
                team2 = lookupTeam(g["MATCHUP"].split(" ")[2])
                home_or_away = ""
                team = ""
                opponent = ""
                if team1 == game["TEAM_ID"]:
                    team = team1
                    opponent = team2
                    if g["MATCHUP"].split(" ")[1] == "@":
                        #team1 is away
                        home_or_away = "away"
                    else:
                        #team1 is home
                        home_or_away = "home"
                elif team2 == game["TEAM_ID"]:
                    team = team2
                    opponent = team1
                    if g["MATCHUP"].split(" ")[1] == "@":
                        #team1 is away
                        home_or_away = "home"
                    else:
                        #team1 is home
                        home_or_away = "away"
                else:
                    print("ERROR: Team not found")
                margin = 0
                if boxscorefinder["TeamStats"][0]["TEAM_ID"] == team:
                    margin = boxscorefinder["TeamStats"][0]["PTS"] - boxscorefinder["TeamStats"][1]["PTS"]
                else:
                    margin = boxscorefinder["TeamStats"][1]["PTS"] - boxscorefinder["TeamStats"][0]["PTS"]
                win_or_loss="win"
                if margin < 0:
                    win_or_loss="loss"
                game_data = {
                    "assists":game["AST"], 
                    "date": g["GAME_DATE"],
                    "margin": margin,
                    "win_or_loss":win_or_loss,
                    "defensive_rebounds":game["DREB"], 
                    "field_goal_percentage":game["FG_PCT"], 
                    "field_goals_attempted":game["FGA"], 
                    "field_goals_made":game["FGM"], 
                    "free_throws_attempted":game["FTA"],
                    "free_throws_made":game["FTM"], 
                    "free_throws_percentage":game["FT_PCT"],
                    "gameID":game["GAME_ID"],
                    "home_or_away":home_or_away,
                    "minutes": game["MIN"],
                    "offensive_rebounds":game["OREB"], 
                    "opponent":opponent,
                    "personal_fouls":game["PF"], 
                    "player":game["PLAYER_ID"],
                    "blocks":game["BLK"],
                    "steals":game["STL"],
                    "points": game["PTS"],
                    "season": season,
                    "team": team,
                    "three_point_percentage":game["FG3_PCT"], 
                    "three_pointers_attempted":game["FG3A"], 
                    "three_pointers_made":game["FG3M"], 
                    "total_rebounds":game["REB"], 
                    "turnovers":game["TO"], 
                    "playoffs": playoffs,
                    "lastModifiedDate":lastModifiedDate
                }
                # print("getting advanced stats")
                for advanced_game in advanced["PlayerStats"]:
                    if game_data['player'] == advanced_game["PLAYER_ID"]:
                        game_data["assist_percentage"] = advanced_game["AST_PCT"]
                        game_data["defensive_rebound_percentage"] = advanced_game["DREB_PCT"]
                        game_data["effective_field_goal_percentage"] = advanced_game["EFG_PCT"]
                        game_data["offensive_rebound_percentage"] = advanced_game["OREB_PCT"]
                        game_data["true_shooting_percentage"] = advanced_game["TS_PCT"]
                        game_data["usage"] = advanced_game["USG_PCT"]
                for misc_game in misc["sqlPlayersMisc"]:
                    if game_data['player'] == misc_game["PLAYER_ID"]:
                        game_data["personal_fouls_drawn"] = misc_game["PFD"]
                # print("inserting player stats")
                db.games.replace_one({"player": game_data["player"], "gameID":game_data["gameID"]}, game_data, True)
                # print("inserted game")
    except Exception as e:
        print("\n ERROR: ", e)
        print(g)

In [133]:
#REGULAR SEASON
attempt = 0
for attempt in range(5):
    print("attempt: " + str(attempt))
    try:
        teams_found = db.teams.find()
        team_count = 0
        for team in teams_found:
            team_count+=1
            print()
            print(str(team_count), "From team: ", team["name"])
            time.sleep(sleeptime)
            playoffs=False
            gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=team["teamID"], league_id_nullable="10", season_type_nullable="Regular Season")
            games = gamefinder.get_normalized_dict()["LeagueGameFinderResults"]
            print("  ", len(games), " total games")
            games = [game for game in games if int(game["SEASON_ID"][1:]) == 2022]
            print("  ", len(games), " recent games")
            found_games = [game for game in db.teamgames.find({"teamID":team["teamID"]})]
            print("  ", len(found_games), " games found in db")
            games = [game for game in games if game_exists(found_games, game["GAME_ID"]) == False]
            print("  ", len(games), " games not in db")

            count=0
            for g in games:
                count+=1
                get_game_data(g, count, playoffs)
        break
    except Exception as e:
        print("failed on attempt: " + str(attempt))
        print(e)
        # print(game)
        time.sleep(sleeptime*10)

attempt: 0

1 From team:  Aces
   839  total games
   18  recent games
   23  games found in db
   0  games not in db

2 From team:  Sun
   781  total games
   19  recent games
   23  games found in db
   0  games not in db

3 From team:  Sky
   548  total games
   18  recent games
   28  games found in db
   0  games not in db

4 From team:  Mystics
   811  total games
   21  recent games
   21  games found in db
   0  games not in db

5 From team:  Dream
   480  total games
   18  recent games
   18  games found in db
   0  games not in db

6 From team:  Wings
   810  total games
   19  recent games
   20  games found in db
   0  games not in db

7 From team:  Storm
   746  total games
   18  recent games
   19  games found in db
   0  games not in db

8 From team:  Sparks
   837  total games
   18  recent games
   18  games found in db
   0  games not in db

9 From team:  Liberty
   837  total games
   18  recent games
   19  games found in db
   0  games not in db

10 From team:  F

In [134]:
#PLAYOFFS
attempt = 0
for attempt in range(5):
    print("attempt: " + str(attempt))
    try:
        teams_found = db.teams.find()
        team_count = 0
        for team in teams_found:
            team_count+=1
            print()
            print(str(team_count), "From team: ", team["name"])
            time.sleep(sleeptime)
            gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=team["teamID"], league_id_nullable="10", season_type_nullable="PlayIn")
            playins = gamefinder.get_normalized_dict()["LeagueGameFinderResults"]
            gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=team["teamID"], league_id_nullable="10", season_type_nullable="Playoffs")
            games = gamefinder.get_normalized_dict()["LeagueGameFinderResults"]
            games = games + playins
            playoffs=True
            print(len(games), " total games")
            games = [game for game in games if int(game["SEASON_ID"][1:]) >= 2021]
            print(len(games), " recent games")
            found_games = [game for game in db.teamgames.find({"teamID":team["teamID"]})]
            print(len(found_games), " games found in db")
            games = [game for game in games if game_exists(found_games, game["GAME_ID"]) == False]
            print(len(games), " games not in db")

            count=0
            for g in games:
                count+=1
                get_game_data(g, count, playoffs)
        break
    except Exception as e:
        print("failed on attempt: " + str(attempt))
        print(e)
        # print(game)
        time.sleep(sleeptime*10)

attempt: 0

1 From team:  Aces
51  total games
5  recent games
23  games found in db
0  games not in db

2 From team:  Sun
62  total games
4  recent games
23  games found in db
0  games not in db

3 From team:  Sky
32  total games
10  recent games
28  games found in db
0  games not in db

4 From team:  Mystics
48  total games
0  recent games
21  games found in db
0  games not in db

5 From team:  Dream
38  total games
0  recent games
18  games found in db
0  games not in db

6 From team:  Wings
54  total games
1  recent games
20  games found in db
0  games not in db

7 From team:  Storm
58  total games
1  recent games
19  games found in db
0  games not in db

8 From team:  Sparks
90  total games
0  recent games
18  games found in db
0  games not in db

9 From team:  Liberty
64  total games
1  recent games
19  games found in db
0  games not in db

10 From team:  Fever
68  total games
0  recent games
20  games found in db
0  games not in db

11 From team:  Mercury
86  total games
11  rec